## Assignment4
Task 1: Install Apache Spark and PySpark on your computer.


Task 2: Collect the data of all NBA players since 1980 (i.e. the year of introduction of 3-point-throws).


Task 3: By making use of PySpark:
 * find best scorers sorted in ascending order,

  *analyze the average number of 3 point attempts for each season in a time limit of 48 minutes (an interval corresponding to an approximate full NBA game),
  
  *predict the average number of 3 point attempts for the next 5 years.

### Task 1: ApacheSpark and PySpark installation completed 

In [1]:
import findspark
findspark.init()
import pyspark
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression



In [2]:
!java -version

java version "21.0.2" 2024-01-16 LTS
Java(TM) SE Runtime Environment (build 21.0.2+13-LTS-58)
Java HotSpot(TM) 64-Bit Server VM (build 21.0.2+13-LTS-58, mixed mode, sharing)


In [3]:
# Initialize or retrieve the existing SparkContext
# SparkContext is the core component for connecting to a Spark cluster
sc = pyspark.SparkContext.getOrCreate()
# Create a SparkSession using the existing SparkContext
spark = SparkSession(sc)

### Task 2: Collect the data of all NBA players since 1980(i.e.the year of introduction of 3-point-throws).

In [4]:
data1 = spark.read.csv('cleaned_data.csv', sep=',', encoding='UTF-8', comment=None, header=True, inferSchema=True)
# Filter rows where the 'Year' is greater than or equal to 1980
data1 = data1.filter(data1.Year >= 1980)
# Select only the specified columns
data1 = data1.select(['Year', 'Player', 'MP', 'G', '3P', '3PA', 'PTS'])
# Display the results
data1.show()

+------+-----------------+----+---+---+---+----+
|  Year|           Player|  MP|  G| 3P|3PA| PTS|
+------+-----------------+----+---+---+---+----+
|1991.0|   Alaa Abdelnaby| 6.7| 43|0.0|0.0| 3.1|
|1991.0|      Danny Ainge|21.4| 80|1.3|3.1|11.1|
|1991.0|      Mark Bryant|14.7| 53|0.0|0.0| 5.1|
|1991.0|     Wayne Cooper|11.1| 67|0.0|0.0| 2.2|
|1991.0|     Walter Davis|20.9| 71|0.2|0.5|13.0|
|1991.0|    Clyde Drexler|34.8| 82|0.7|2.3|21.5|
|1991.0|  Kevin Duckworth|31.0| 81|0.0|0.0|15.8|
|1991.0|    Jerome Kersey|32.3| 73|0.1|0.2|14.8|
|1991.0|     Terry Porter|32.9| 81|1.6|3.9|17.0|
|1991.0|Clifford Robinson|23.7| 82|0.1|0.2|11.7|
|1991.0|    Buck Williams|32.3| 80|0.0|0.0|11.7|
|1991.0|      Danny Young|12.0| 75|0.5|1.4| 3.8|
|1992.0|   Alaa Abdelnaby|13.2| 71|0.0|0.0| 6.1|
|1992.0|      Danny Ainge|19.7| 81|1.0|2.8| 9.7|
|1992.0|      Mark Bryant|14.3| 56|0.0|0.1| 4.1|
|1992.0|     Wayne Cooper| 9.8| 35|0.0|0.0| 2.2|
|1992.0|    Clyde Drexler|36.2| 76|1.5|4.4|25.0|
|1992.0|  Kevin Duck

Notes:after filtering and selecting specific columns,we get the representation of the DataFrame data1,Each row corresponds to a different player's statistics for a specific year,Spark DataFrame displays the top 20 rows by default, providing a glimpse of the dataset.

This output helps visualize the structured data and allows for initial analysis of player performance metrics over the specified years.

In [5]:
##The schema defines the data types and structure of each column in the DataFrame, it informs how data can be accessed, transformed, and queried 
data1.printSchema()

root
 |-- Year: double (nullable = true)
 |-- Player: string (nullable = true)
 |-- MP: double (nullable = true)
 |-- G: integer (nullable = true)
 |-- 3P: double (nullable = true)
 |-- 3PA: double (nullable = true)
 |-- PTS: double (nullable = true)



Year: DoubleType (nullable = true) - Represents the year when the statistics were recorded. It's of type double, meaning it can store fractional values.
Player: StringType (nullable = true) - Stores the name of the basketball player. It's of type string.
MP: DoubleType (nullable = true) - Indicates the average minutes played per game. It's of type double.
G: IntegerType (nullable = true) - Represents the number of games played in the season. It's of type integer.
3P: DoubleType (nullable = true) - Shows the average number of 3-point field goals made per game. It's of type double.
3PA: DoubleType (nullable = true) - Displays the average number of 3-point field goals attempted per game. It's of type double.
PTS: DoubleType (nullable = true) - Represents the average points scored per game. It's of type double.

### By making use of PySpark: find best scorers sorted in ascending order,

1. TOP 10 scores sorted in ascending order

In [6]:
top = data1.sort(col("PTS").asc())
top.collect()[-10:]

[Row(Year=1993.0, Player='Michael Jordan', MP=39.3, G=78, 3P=1.0, 3PA=2.9, PTS=32.6),
 Row(Year=1985.0, Player='Bernard King', MP=37.5, G=55, 3P=0.0, 3PA=0.2, PTS=32.9),
 Row(Year=2006.0, Player='Allen Iverson', MP=43.1, G=72, 3P=1.0, 3PA=3.1, PTS=33.0),
 Row(Year=1980.0, Player='George Gervin', MP=37.6, G=78, 3P=0.4, 3PA=1.3, PTS=33.1),
 Row(Year=1990.0, Player='Michael Jordan', MP=39.0, G=82, 3P=1.1, 3PA=3.0, PTS=33.6),
 Row(Year=2020.0, Player='James Harden', MP=36.5, G=68, 3P=4.4, 3PA=12.4, PTS=34.3),
 Row(Year=1988.0, Player='Michael Jordan', MP=40.4, G=82, 3P=0.1, 3PA=0.6, PTS=35.0),
 Row(Year=2006.0, Player='Kobe Bryant', MP=41.0, G=80, 3P=2.3, 3PA=6.5, PTS=35.4),
 Row(Year=2019.0, Player='James Harden', MP=36.8, G=78, 3P=4.8, 3PA=13.2, PTS=36.1),
 Row(Year=1987.0, Player='Michael Jordan', MP=40.0, G=82, 3P=0.1, 3PA=0.8, PTS=37.1)]

This operation will give us the top 10 players with the highest average points scored per game, as the DataFrame was sorted in ascending order and we are accessing the last 10 rows of that sorted DataFrame.for exmple Michael Jordan (1993): Played 78 games with an average of 39.3 minutes per game (MP), scored 32.6 points per game (PTS), made 1.0 three-pointers (3P) per game on 2.9 attempts (3PA).



2. analyze the average number of 3 point attempts for each season in a time limit of 48minutes (an interval corresponding to an approximate full NBAgame),

 year is equivalent to a season based on the data description. NBA has 5 players per team, which implies 10players in 2teams. To get the number of games in each seasons we divide the total time spent on a field by all players by 480(assuming an average game length of 48 minutes and 10 players on the court at any time)..


In [7]:
period = data1.groupBy('Year').agg((sum('MP')/(10)).alias('Time_per_a_Player'), (sum('MP')/480).alias('No_of_games')).orderBy(desc('Year'))
period.show(10)

+------+------------------+------------------+
|  Year| Time_per_a_Player|       No_of_games|
+------+------------------+------------------+
|2021.0| 1072.149999999999| 22.33645833333331|
|2020.0|1032.2399999999993| 21.50499999999999|
|2019.0|1026.1700000000005|21.378541666666674|
|2018.0| 1038.249999999999|21.630208333333314|
|2017.0| 972.3799999999999|20.257916666666667|
|2016.0| 965.0999999999997|20.106249999999992|
|2015.0|1002.2599999999991|20.880416666666648|
|2014.0| 979.4699999999988|20.405624999999976|
|2013.0| 961.7999999999995|20.037499999999987|
|2012.0| 997.3699999999992|20.778541666666648|
+------+------------------+------------------+
only showing top 10 rows



Year 2021 had the highest average player time per game, indicating potentially longer game durations or higher player involvement.
Years 2018 to 2020 also show relatively high player time per game, suggesting intensive game seasons during those years.
The number of games fluctuates slightly year over year, reflecting variations in season schedules and player availability.

The average number of 3 point attempts for each season in a time limit of 48 minutes (an interval corresponding to an approximate
full NBA game).

In [8]:
All_3Attempts = data1.groupBy('Year').agg(sum('3PA').alias('total3PA')).orderBy(desc('Year'))
All_3Attempts.show(10)

+------+------------------+
|  Year|          total3PA|
+------+------------------+
|2021.0|1522.4999999999986|
|2020.0|1427.7000000000005|
|2019.0|1341.0999999999995|
|2018.0|1240.0000000000002|
|2017.0|1068.3999999999999|
|2016.0|  941.300000000001|
|2015.0| 937.7000000000015|
|2014.0| 863.2000000000004|
|2013.0| 794.6000000000008|
|2012.0| 760.1000000000008|
+------+------------------+
only showing top 10 rows



In [9]:
from pyspark.sql.functions import col, sum

# Calculate average 3-point attempts per season in a 48-minute interval
#average_3pa_per_season = data1.groupBy('Year') \
    #.agg((sum('3PA') / (sum('MP') / 480)).alias('Avg_3PA_per_48mins')) \
    #.orderBy('Year')

#average_3pa_per_season.show()


In [10]:
All_3Attempts.select("total3PA").show()

+------------------+
|          total3PA|
+------------------+
|1522.4999999999986|
|1427.7000000000005|
|1341.0999999999995|
|1240.0000000000002|
|1068.3999999999999|
|  941.300000000001|
| 937.7000000000015|
| 863.2000000000004|
| 794.6000000000008|
| 760.1000000000008|
| 686.8000000000004|
| 702.8000000000011|
| 696.1000000000007|
| 693.1000000000006|
| 655.3000000000005|
|  608.700000000001|
| 618.2000000000005|
| 551.2000000000005|
| 520.3000000000003|
| 547.9000000000003|
+------------------+
only showing top 20 rows



showing the average 3-point attempts per 48 minutes for each year

In [11]:
##create a new DataFrame called game_stat which joins two DataFrames (period and All_3Attempts) based on the 'Year' column and selects specific columns
game_stat = period.join(All_3Attempts, All_3Attempts['Year'] == period['Year']).select(period['*'], All_3Attempts['total3PA'])
game_stat.show()

+------+------------------+------------------+------------------+
|  Year| Time_per_a_Player|       No_of_games|          total3PA|
+------+------------------+------------------+------------------+
|1988.0| 704.5000000000002|14.677083333333337|148.79999999999993|
|1987.0| 713.4800000000004|14.864166666666675|137.99999999999986|
|2010.0| 943.4399999999989|19.654999999999976| 702.8000000000011|
|1993.0| 832.5299999999997|17.344374999999996|297.79999999999995|
|2001.0| 920.9200000000008| 19.18583333333335| 499.4000000000006|
|1984.0| 662.6000000000006|13.804166666666678| 71.10000000000004|
|1980.0| 651.5499999999998| 13.57395833333333| 71.79999999999997|
|1997.0| 930.8900000000003|19.393541666666675| 627.9000000000004|
|1992.0| 819.8000000000004|17.079166666666673|261.29999999999956|
|1990.0| 809.1000000000003|16.856250000000006|219.19999999999976|
|1995.0| 866.5400000000009|18.052916666666686| 536.0000000000008|
|2009.0| 943.3199999999995| 19.65249999999999| 696.1000000000007|
|2007.0| 9

Each row in the DataFrame corresponds to a specific year and includes aggregated metrics related to player time, game count, and 3-point attempts. The values in each column provide insights into the average player involvement, game frequency, and the prevalence of 3-point attempts across different years in the dataset.

Predict the avearge number of 3 points attempts for the next 5 years

In [12]:
train = game_stat.rdd.map(lambda y: [Vectors.dense(y[0]), y[-1]]).toDF(['features', 'avg3PA']) 
train.show(5)

+--------+------------------+
|features|            avg3PA|
+--------+------------------+
|[1988.0]|148.79999999999993|
|[1987.0]|137.99999999999986|
|[2010.0]| 702.8000000000011|
|[1993.0]|297.79999999999995|
|[2001.0]| 499.4000000000006|
+--------+------------------+
only showing top 5 rows



In [13]:
# Now we train Linear Regression model on game_stat
algorithm = LinearRegression(featuresCol="features", labelCol="avg3PA")
# Fit the model
model = algorithm.fit(train)
## Print coefficients
print("Coefficient: " + str(model.coefficients))

Coefficient: [30.001499068072476]


for   showing the actual avg3PA values and the predicted values  based on the features (Year, Time_per_a_Player, No_of_games) in  training dataset (train):

In [14]:
# Prediction
predictions = model.transform(train)
predictions.show(5)

+--------+------------------+------------------+
|features|            avg3PA|        prediction|
+--------+------------------+------------------+
|[1988.0]|148.79999999999993|178.94554736337886|
|[1987.0]|137.99999999999986|148.94404829530686|
|[2010.0]| 702.8000000000011| 838.9785268609776|
|[1993.0]|297.79999999999995| 328.9530427037389|
|[2001.0]| 499.4000000000006| 568.9650352483222|
+--------+------------------+------------------+
only showing top 5 rows

